In [1]:
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import json
import datetime

import string 
import re

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
%load_ext tensorboard

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\melli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\melli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Init file
words = []
classes = []
documents = []

data_file = open("intents.json").read()
intents = json.loads(data_file)

# init stop words
list_stopwords = set(stopwords.words('indonesian'))

# init stem
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# words
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # lowercase pattern
        pattern = pattern.lower()

        # remove punctuation
        pattern = pattern.translate(str.maketrans("","",string.punctuation))
        
        #remove whitespace leading & trailing
        pattern = pattern.strip()
        
        #remove multiple whitespace into single whitespace
        pattern = re.sub('\s+',' ',pattern)
        
        # take each word and tokenize it
        tokens = word_tokenize(pattern)
        words.extend(tokens)

        # remove stop words
        words = [w for w in words if not w in list_stopwords]
        
        # stem
        words = [stemmer.stem(w) for w in words]
        
        # adding documents
        documents.append((tokens, intent["tag"]))

        # adding classes to our class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# print(len(documents), "documents")
# print(len(classes), "classes", classes)
# print(len(words), "unique lemmatized words", words)

# save as pickle
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

# init training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # init bag of words
    bag = []

    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # create bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")


# model training
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

learn_rate=.001
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# compile model

model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

# fit and save the model
Epochs = 100
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)
set_callback = EarlyStopping(patience=3,
                                            restore_best_weights=True, verbose=2)

history = model.fit(np.array(train_x), np.array(train_y), batch_size=16,
                      callbacks=[tensorboard_callback, set_callback], epochs=Epochs,verbose=1)


model.save("chatbot_model.h5", history)
print("model created")

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_6056\1206607519.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Training data created
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               81408     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 12)                780       
                                                                 
Total params: 90,444
Trainable params: 90,444
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


C:\Users\LENOVO\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


51/51 [==============================] - 1s 4ms/step - loss: 2.4190 - accuracy: 0.1936
Epoch 2/100
51/51 [==============================] - 0s 4ms/step - loss: 2.1601 - accuracy: 0.2868
Epoch 3/100
51/51 [==============================] - 0s 4ms/step - loss: 1.8733 - accuracy: 0.3480
Epoch 4/100
51/51 [==============================] - 0s 4ms/step - loss: 1.5968 - accuracy: 0.4841
Epoch 5/100
51/51 [==============================] - 0s 4ms/step - loss: 1.3465 - accuracy: 0.5858
Epoch 6/100
51/51 [==============================] - 0s 4ms/step - loss: 1.1176 - accuracy: 0.6605
Epoch 7/100
51/51 [==============================] - 0s 4ms/step - loss: 0.9805 - accuracy: 0.7034
Epoch 8/100
51/51 [==============================] - 0s 4ms/step - loss: 0.8201 - accuracy: 0.7463
Epoch 9/100
51/51 [==============================] - 0s 4ms/step - loss: 0.7526 - accuracy: 0.7782
Epoch 10/100
51/51 [==============================] - 0s 4ms/step - loss: 0.6546 - accuracy: 0.8064
Epoch 11/100
51/51 [=

In [ ]:
# Export the Model
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

savedmodel = "word_model.h5"
tf.saved_model.save(model, savedmodel)
loaded = tf.saved_model.load(savedmodel)

converter = tf.lite.TFLiteConverter.from_saved_model(savedmodel)
tflite_model = converter.convert()

tflite_model_file = 'model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)
    
print("model created")